## Proyecto Final Hackio - Informe temporada 2015/2016 Athletic Club

#### Extracción Datos

- Eventos partidos (statsbomb)

In [ ]:
# Importación librerías

from statsbombpy import sb
import pandas as pd
import numpy as np
import time

# Extracción datos desde API statsbomb

competiciones = sb.competitions() # Competiciones disponibles
df_LaLiga = competiciones[competiciones["competition_id"] == 11] # Filtro por La Liga

# Conseguir el ID de cada partido
df_partidos = pd.DataFrame()
for x in df_LaLiga["season_id"]:
    df_temporal = sb.matches(competition_id=11, season_id = x)
    df_partidos = pd.concat([df_partidos, df_temporal], ignore_index=True)
liga2015 = df_partidos[df_partidos["season"] == "2015/2016"]
lista_2015 = []
for x in liga2015["match_id"]:
    lista_2015.append(x)

# Extraer los eventos de cada partido
df_eventos_liga2015 = pd.DataFrame() # Crear DataFrame
errores = [] # Aculumar los Ids de partido que puedan dar fallo para recuperarlos posteriormente
for x in lista_2015: # Iterar sobre la lista de partidos para extraer cada Id
    try:
        evento= sb.events(match_id=x)
        df_temp = evento # Crear un df temporal con los eventos del partido
        df_eventos_liga2015 = pd.concat([df_eventos_liga2015, df_temp], ignore_index=True) # Concatenar con df para guardar el partido
        time.sleep(0.5)
        time.sleep(0.5)
    except Exception as e:
        errores.append(x)

# Guardar archivo
df_eventos_liga2015.to_csv("../Data/df_eventos_liga2015.csv", index=False)

* Información jugadores (API Transfermarkt)

In [ ]:
# Importar librerías
import pandas as pd
import requests
import os
from dotenv import load_dotenv
load_dotenv()

# Extraer la tabla de equipos
url = "https://transfermarket.p.rapidapi.com/competitions/get-table"
rapidapi_key_1 = os.getenv("rapidapi-key_1")
querystring = {"id":"ES1","seasonID":"2015","domain":"es"}
headers = {
	"x-rapidapi-key": rapidapi_key_1,
	"x-rapidapi-host": "transfermarket.p.rapidapi.com"
}
response = requests.get(url, headers=headers, params=querystring)
# Almacenar resultados en variable
tabla = response.json()
# Extraer ids de equipo dentro de la tabla
lista_ids = []
for x in tabla["table"]:
     lista_ids.append(x["id"])
# Extraer los equipos
equipos = {}
for x in lista_ids:

    url = "https://transfermarket.p.rapidapi.com/clubs/get-squad"

    querystring = {"id": str(x),"saison_id":"2015","domain":"es"}

    headers = {
        "x-rapidapi-key": rapidapi_key_1,
        "x-rapidapi-host": "transfermarket.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    equipos[x] = response.json()

# Extraer cada jugador
lista_jug_ids = []
for x, i in equipos.items():
    for x, i in i.items():
        for x in i:
            lista_jug_ids.append(x["id"])

jugadores = {}
for x in lista_jug_ids:

    url = "https://transfermarket.p.rapidapi.com/players/get-profile"

    querystring = {"id":str(x),"domain":"es"}

    headers = {
        'x-rapidapi-key': rapidapi_key_1,
        'x-rapidapi-host': "transfermarket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    jugadores[x] = response.json()
    if response.status_code == 200:
        print(f"añadido {x}")
    else:
        print(response.status_code)

no_añadidos = []
for a, b in jugadores.items():
    for c, d in b.items():
        if c == "message":
            no_añadidos.append(a)

for x in no_añadidos:

    url = "https://transfermarket.p.rapidapi.com/players/get-profile"

    querystring = {"id":str(x),"domain":"es"}

    headers = {
        'x-rapidapi-key': rapidapi_key_1,
        'x-rapidapi-host': "transfermarket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    jugadores[x] = response.json()
    if response.status_code == 200:
        print(f"añadido {x}")
    else:
        print(response.status_code)

# Añadir aquellos que han dado fallo
for x in no_añadidos:

    url = "https://transfermarket.p.rapidapi.com/players/get-profile"
    rapidapi_key_2 = os.getenv("rapidapi_key_2")
    querystring = {"id":str(x),"domain":"es"}

    headers = {
        'x-rapidapi-key': rapidapi_key_2,
        'x-rapidapi-host': "transfermarket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    jugadores[x] = response.json()
    if response.status_code == 200:
        print(f"añadido {x}")
    else:
        print(response.status_code)

# Almacenar los datos
data = []
for jugador_id, info in jugadores.items():
    perfil = info.get("playerProfile", {})
    data.append(perfil)

# Crear el DataFrame
df_jugadores = pd.DataFrame(data)

# Guardar archivo
df_jugadores.to_csv("data/jugadores_info.csv", index=False)

* Datos equipos (API FBREF)

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

# Conseguir los IDs de equipo para API
lista_ligas = {"2015-2016": "https://fbref.com/es/comps/12/2015-2016/Estadisticas-2015-2016-La-Liga"}
lista_equipos = {}
for a, x in lista_ligas.items():
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    url = x
    driver.get(url)
    driver.implicitly_wait(10)

    # Buscar todos los enlaces de equipos
    tabla = driver.find_element(By.CSS_SELECTOR, "table.stats_table.sortable.min_width.force_mobilize.now_sortable")
    enlaces = tabla.find_elements(By.CSS_SELECTOR, 'a[href*="/es/equipos/"]')

    equipos = []

    for enlace in enlaces:
        href = enlace.get_attribute("href")
        texto = enlace.text.strip()
        
        partes = href.split("/")

        try:
            idx = partes.index("equipos")
            id_equipo = partes[idx + 1]  # El ID está justo después de "equipos"
            equipos.append(id_equipo)
        except (ValueError, IndexError):
            continue  # Si no encuentra "equipos" o el ID está mal formado
    driver.quit()
    time.sleep(5)
    lista_equipos[a] = equipos

# Generar API-Key
response = requests.post('https://fbrapi.com/generate_api_key')
api_key = response.json()['api_key']

# Conseguir información de cada equipo
url = "https://fbrapi.com/teams"
jugadores = {}
no_añadidos = {}
headers = {"X-API-Key": api_key}

for temporada, lista_ids in lista_equipos.items():  # lista_ids es una lista de team_ids
    for team_id in lista_ids:
        params = {
            "team_id": team_id,
            "season_id": temporada
        }
        try:
            response = requests.get(url, params=params, headers=headers)
            print(f"Equipo {team_id}, Año {temporada}, Status: {response.status_code}")

            if response.status_code == 200:
                if temporada not in jugadores:
                    jugadores[temporada] = {}  # lista de equipos para esa temporada
                jugadores[temporada][team_id] = response.json()
            else:
                if temporada not in no_añadidos:
                    no_añadidos[temporada] = {}
                no_añadidos[temporada].append(team_id)

        except Exception as e:
            print(f"Error en temporada {temporada}, equipo {team_id}: {e}")
            if temporada not in no_añadidos:
                no_añadidos[temporada] = {}
            no_añadidos[temporada].append(team_id)
        time.sleep(6)  # por si hay límite de llamadas

# Guardar jugadores
df_jugadores = []
for x, i in jugadores.items():  # x = temporada, i = diccionario de equipos
    for a, b in i.items():      # a = team_id, b = contenido (team_roster, team_schedule)
        c = b["team_roster"]
        for d, e in c.items():  # c debería ser "data"
            for f in e:  # e = jugador (diccionario)
                f["season_id"] = x
                f["team_id"] = a
                df_jugadores.append(f)

# Crear el DataFrame
df_jugadores = pd.DataFrame(df_jugadores)  

# Extraer estadísticas de temporada de cada jugador
url = "https://fbrapi.com/player-season-stats"
headers = {"X-API-Key": api_key}
data_jugadores = {}
for x, i in lista_equipos.items():
    for team_id in i:
        params = {
        "team_id": team_id,
        "league_id": 12,
        "season_id": "2015-2016"}
        response = requests.get(url, params=params, headers=headers)
        if response.status_code == 200:
            data_jugadores[team_id] = response.json()
        time.sleep(3)
for team_id, team_info in data_jugadores.items():
    print(team_info)
    for player in team_info['players']:
        print(player)
# Lista para almacenar los datos de cada jugador
players_data = []

# Iteramos sobre cada equipo en el diccionario
for team_id, team_info in data_jugadores.items():
    # Iteramos sobre cada jugador en el equipo
    for player in team_info['players']:
        # Creamos un diccionario base con el ID del equipo
        player_dict = {'team_id': team_id}
        
        # Añadimos todos los campos de meta_data con prefijo 'meta_'
        for key, value in player['meta_data'].items():
            player_dict[f'{key}'] = value
        
        # Añadimos todos los campos de stats.stats con prefijo 'stats_'
        for key, value in player['stats']['stats'].items():
            player_dict[f'{key}'] = value
        
        # Añadimos todos los campos de stats.playingtime con prefijo 'playingtime_'
        for key, value in player['stats']['playingtime'].items():
            player_dict[f'{key}'] = value
        
        # Añadimos este jugador a la lista
        players_data.append(player_dict)

# Creamos el DataFrame
df_info = pd.DataFrame(players_data)
# Eliminamos duplicados
df_info = df_info.dropna(subset=['player_id'])

# Guardamos archivo
df_info.to_csv("data/jugadores_LaLiga_completo.csv", index = False)

* Clasificación

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import re
from statsbombpy import sb
import src as src
from src import normalizacion_nombres

# Crear URLs para cada jornada
contador = 0
plantilla_url = "https://www.ceroacero.es/edicion/campeonato-espanhol-2015-16/87618?jornada_in={x}&fase=82379"
urls_por_jornada = {}
for contador in range(0,38):
    url = plantilla_url.format(x=contador+1)
    urls_por_jornada[contador+1] = url

# Configuración del navegador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
df_jornadas = pd.DataFrame()
for x, i in urls_por_jornada.items():
    # URL
    base_url = i
    driver.get(base_url)
    datos = []
    encabezado = []
    # Espera
    wait = WebDriverWait(driver, 10)
    # Clic en el botón de cookies
    try:
        cookie_button = driver.find_element(By.CSS_SELECTOR, ".fc-button-label")
        cookie_button.click()
    except:
        pass  # En caso de que no aparezca seguir con el código
    try:
        cookie_button2 = driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16")
        cookie_button2.click()
    except:
        pass  # En caso de que no aparezca seguir con el código

    # Buscar clasificacion
    tabla_element = wait.until(EC.presence_of_element_located((By.ID, "DataTables_Table_0")))
    if not encabezado: # Solo la primera vez
        headers_th = tabla_element.find_elements(By.XPATH, ".//thead/tr/th")
        # Priorizar el texto del 'aria-label' si el texto normal está vacío o es genérico
        # O simplemente tomar el texto visible
        for th in headers_th:
            aria_label = th.get_attribute("aria-label")
            text = th.text.strip()
            if text and text not in [":", ""]: # Usar texto si es significativo
                encabezado.append(text)
            elif aria_label and ":" in aria_label: # Procesar aria-label
                    # "Pts: activate to sort column ascending" -> "Pts"
                encabezado.append(aria_label.split(":")[0].strip())
            elif text: # Fallback a texto si aria-label no es útil
                encabezado.append(text)
            else: # Si ambos son malos, un placeholder o intentar inferir
                encabezado.append(f"Col_{len(encabezado)+1}") # Placeholder

        # Limpieza de posibles columnas vacías o no deseadas del final si tienen placeholders
        if encabezado and encabezado[-1].startswith("Col_"):
            # La última columna parece ser de acciones, la removemos si no tiene un buen nombre
                is_action_col = tabla_element.find_elements(By.XPATH, ".//tbody/tr[1]/td[last()]/a/span[contains(@class, 'icn_zerozero')]")
                if is_action_col:
                    encabezado.pop()

        # Asegurarse de que los nombres de las columnas sean únicos si es necesario
        # y que coincidan con el número de celdas que planeas extraer.
        # Por la estructura de la tabla HTML, el nombre del equipo está en la 3ra columna visual,
        # pero la 2da <th> puede estar vacía (para el logo).
        # Ajuste manual si es necesario, basado en la inspección visual de la tabla:
        # Ejemplo de ajuste si los nombres automáticos no son perfectos:
        expected_cols = ["Pos", "Logo", "Equipo", "Pts", "P", "V", "E", "D", "GM", "GR", "DG"] # Menos la última de acción
        if len(encabezado) >= len(expected_cols):
                # La primera columna <th> es para Pos, la segunda para Logo (sin texto), la tercera para Equipo (texto vacío)
                # Usaremos una lista predefinida para mayor robustez con esta tabla específica.
                column_headers = expected_cols


    # Extraer filas de datos
    filas = tabla_element.find_elements(By.XPATH, ".//tbody/tr")
    #print(f"Encontradas {len(filas)} filas en Jornada {i+1}.")

    for fila in filas:
        celdas = fila.find_elements(By.TAG_NAME, "td")
        if len(celdas) >= len(column_headers): # Asegurar que hay suficientes celdas
            # Extraer el texto de cada celda. Adaptar según la estructura.
            # La columna del equipo (índice 2) tiene un enlace <a>
            posicion = celdas[0].text.strip()
            # logo_img_src = celdas[1].find_element(By.TAG_NAME, "img").get_attribute("src") # Opcional
            equipo_link = celdas[2].find_element(By.TAG_NAME, "a")
            equipo = equipo_link.text.strip()
            # equipo_url = equipo_link.get_attribute("href") # Opcional

            # Resto de las celdas (Pts, P, V, E, D, GM, GR, DG)
            # Índices 3 a 10 para los datos si column_headers tiene 11 elementos
            datos_numericos = [celda.text.strip() for celda in celdas[3:len(column_headers)]]


            registro = [posicion, "logo_placeholder", equipo] + datos_numericos
            registro_dict = dict(zip(column_headers, registro))
            registro_dict['Jornada'] = x # Añadir la jornada actual
            datos.append(registro_dict)
        #else:
            #print(f"Fila omitida en Jornada {i+1} por no tener suficientes celdas: {[c.text for c in celdas]}")


    # 5. Convertir a DataFrame de Pandas y guardar (opcional)
    if datos:
        df = pd.DataFrame(datos)
        df_jornadas = pd.concat([df_jornadas, df], ignore_index=True)



# 4. Cerrar el WebDriver
driver.quit()

# Eliminar columnas no necesarias
df_jornadas.drop(columns=(["Logo"]), inplace=True)

# Normalizar los nombres de los equipos y columnas
df_wyscout = pd.read_csv("data/Tabla_Partidos.csv")
clubs_statsbomb = df_wyscout["Local"].unique().tolist()
clubs_jornadas = df_jornadas["Equipo"].unique().tolist()
df_clubes = src.normalizacion_nombres(clubs_statsbomb, clubs_jornadas)
# Crear diccionario
dict_nombres_equipos = dict(zip(df_clubes["Nombre_unificado"], df_clubes["Nombre_original"]))
# Mapear nombres en todas las columnas
df_jornadas["Equipo"] = df_jornadas["Equipo"].map(dict_nombres_equipos)
df_combinado = df_jornadas.merge(
    df_wyscout[["Local", "id_club_local_L"]],
    left_on=["Equipo"],
    right_on=["Local"],
    how="left"
)
df_combinado.drop(columns=(["Local"]), inplace=True)
df_combinado.rename(columns=({"id_club_local_L":"clubid"}), inplace=True)

# Guardar archivo
df_combinado.to_csv("data/clasificacion.csv" ,index=False)

* Lesiones

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import re

# Conseguir cada URL del equipo y sus nombres
lista_equipos_urls = []
equipos_nombres = {}
# Configuración del navegador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
# URL
base_url = "https://www.ceroacero.es/edicion/campeonato-espanhol-2015-16/87618"
driver.get(base_url)
# Espera
wait = WebDriverWait(driver, 10)
# Clic en el botón de cookies
try:
    cookie_button = driver.find_element(By.CSS_SELECTOR, ".fc-button-label")
    cookie_button.click()
except:
    pass  # En caso de que no aparezca seguir con el código
try:
    cookie_button2 = driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16")
    cookie_button2.click()
except:
    pass  # En caso de que no aparezca seguir con el código

# Buscar todos los enlaces de equipos
tabla = driver.find_element(By.CSS_SELECTOR, "table.zztable.stats.zz-datatable.dataTable.no-footer")
enlaces = tabla.find_elements(By.CSS_SELECTOR, 'a[href*="/equipo/"]')
for enlace in enlaces:
    href = enlace.get_attribute("href")
    equipo = href.split("/equipo/")[1].split("?")[0].rstrip("/")
    if equipo not in lista_equipos_urls:
        lista_equipos_urls.append(equipo)
    nombre = enlace.text
    equipos_nombres[equipo] = nombre
driver.quit()

# Crear URL personalizada de cada equipo
plantilla_url = "https://www.ceroacero.es/equipo/{x}/unavailable?epoca_sidelines_id=145"

urls_por_equipo = {}

for equipo in lista_equipos_urls:
    if equipo == "athletic-bilbao":
        equipo = "athletic"
    url = plantilla_url.format(x=equipo)
    urls_por_equipo[equipo] = url


# Conseguir Dataframe final
df_lesiones_equipo = pd.DataFrame()
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
for x, i in urls_por_equipo.items():

    dictio = {"columnas": [], "datos": []}
    driver.get(i)
    # Clic en el botón de cookies
    try:
        driver.find_element(By.CSS_SELECTOR, ".fc-button-label").click()
    except:
        pass
    try:
        driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16").click()
    except:
        pass
    time.sleep(5)
    # Obtener la tabla
    info = driver.find_elements(By.CSS_SELECTOR, "table.zz-datatable.zztable.stats")
    for item in info:
        columnas = [th.text.strip() for th in item.find_elements(By.CSS_SELECTOR, "th")]
        dictio["columnas"].extend(columnas + ["href"])
        filas = item.find_elements(By.CSS_SELECTOR, "tbody tr")
        for fila in filas:
            datos = [td.text.strip() for td in fila.find_elements(By.CSS_SELECTOR, "td")]
            enlace_jugador = fila.find_element(By.CSS_SELECTOR, 'a[href*="/jugador/"]')
            href = enlace_jugador.get_attribute("href")
            datos.append(href)
            dictio["datos"].append(datos)
    # Guardar DataFrame temporal
    if dictio["datos"]:
        df_temporal = pd.DataFrame(dictio["datos"], columns=dictio["columnas"])
        df_temporal["equipo"] = x
        df_lesiones_equipo = pd.concat([df_lesiones_equipo, df_temporal], ignore_index=True)
    print(f"Datos de {x} desde {i}")
driver.quit()

# Hacer diccionario para los nombres completos de cada jugador (href)
df_jugadores_href = df_lesiones_equipo[["JUGADOR", "href"]].drop_duplicates()
diccionario_jugadores = df_jugadores_href.set_index("JUGADOR")["href"].to_dict()

# Hacer diccionario con nombre de cada jugador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
dictio_jugadores = {}
for nombre, url in diccionario_jugadores.items():
    driver.get(url)
    try:
        driver.find_element(By.CSS_SELECTOR, ".fc-button-label").click()
    except:
        pass
    try:
        driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16").click()
    except:
        pass
    time.sleep(2)
    tabla = driver.find_elements(By.CSS_SELECTOR, "#entity_bio.frame")
    nombre_elemento = tabla[0].find_elements(By.CSS_SELECTOR, ".bio")[0]
    jugador1 = nombre_elemento.text.split("NOMBRE")[1]
    jugador = jugador1.split("\n")[1]
    dictio_jugadores[nombre] = jugador
driver.quit()

# Mapear columnas
df_lesiones_equipo["href"] = df_lesiones_equipo["JUGADOR"].map(dictio_jugadores)
df_lesiones_equipo["equipo"] = df_lesiones_equipo["equipo"].map(equipos_nombres)

# Guardar archivo
df_lesiones_equipo.to_csv("data/lesiones_completo.csv", index=False)

#### Normalización Datos

* Jugadores

In [ ]:
# Importar librerías
import pandas as pd
from rapidfuzz import process, fuzz
import src as src
pd.set_option('display.max_columns', None)

# Cargar archivos
df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")
df_jugadores_info = pd.read_csv("data/jugadores_info.csv")

# Cambiar el primer 'Sergio Alvarez' (duplicado)
sergio_idx = df_jugadores_info[df_jugadores_info['playerFullName'] == 'Sergio Álvarez'].index[0]
df_jugadores_info.at[sergio_idx, 'playerFullName'] = 'Sergio Álvarez Conde'
# Cambiar el primer 'Pablo Hernandez' (duplicado)
pablo_idx = df_jugadores_info[df_jugadores_info['playerFullName'] == 'Pablo Hernández'].index[0]
df_jugadores_info.at[pablo_idx, 'playerFullName'] = 'Pedro Pablo Hernández'
# Cambiar el primer 'Pedro López' (duplicado)
pedro_idx = df_jugadores_info[df_jugadores_info['playerFullName'] == 'Pedro López'].index[1]
df_jugadores_info.at[pedro_idx, 'playerFullName'] = 'Pedro López Muñoz'

# Crear listas de los nombres y equipo que aparecen en cada df
df_jugadoresunicos_eventos = df_eventos["player"].unique().tolist()
df_jugadoresunicos_info = df_jugadores_info["playerFullName"].unique().tolist()
df_clubesunicos_eventos = df_eventos["team"].unique().tolist()
df_clubesunicos_info = df_jugadores_info["club"].unique().tolist()

# Normalizar con función
df_nombres = src.normalizacion_nombres(df_jugadoresunicos_info, df_jugadoresunicos_eventos)
df_clubes = src.normalizacion_nombres(df_clubesunicos_info, df_clubesunicos_eventos)
# Crear diccionario para mapear los nombres
dict_nombres_info = df_nombres.set_index('Nombre_original')['Nombre_unificado'].to_dict()
# Mapear nombres jugadores
df_jugadores_info["playerFullName"] = df_jugadores_info["playerFullName"].apply(
    lambda x: dict_nombres_info.get(x, x) if dict_nombres_info.get(x) is not None else x)
# Crear diccionario para mapear los equipos
df_equipos_jugadores = df_eventos[["player", "team"]].drop_duplicates()
dict_equipos_jugadores = dict(zip(df_equipos_jugadores["player"], df_equipos_jugadores["team"]))
# Mapear los equipos en la temporada 2015/2016 ya que la info que aparece es actual
df_jugadores_info["club"] = df_jugadores_info["playerFullName"].map(dict_equipos_jugadores)
# Crear diccionarios para mapear los IDs de jugador y equipo
df_equipos = df_eventos[["team", "team_id"]].drop_duplicates()
dict_equipos = dict(zip(df_equipos["team"], df_equipos["team_id"]))
df_jugadores = df_eventos[["player", "player_id"]].drop_duplicates()
dict_jugadores = dict(zip(df_jugadores["player"], df_jugadores["player_id"]))
# Mapear ID Jugadores
df_jugadores_info["playerID"] = df_jugadores_info["playerFullName"].map(dict_jugadores)
# Mapear ID Equipos
df_jugadores_info["clubID"] = df_jugadores_info["club"].map(dict_equipos)
# Eliminar columnas no necesarias
df_jugadores_info.drop(columns=(['player_name', 'team_id', 'player_country_code', 'age',
       'positions', 'unused_sub',
       'team_gls_on_pitch', 'team_gls_ag_on_pitch', 'per90_plus_minus',
       'per90_on_off']), inplace=True)
# Cargar fichero para unir df
df_tabla = pd.read_csv("data/jugadores_info_normalizado.csv")
df_combinado = df_tabla.merge(
    df_jugadores_info,
    left_on=["playerID"],
    right_on=["player_id"],
    how="left"
)
# Eliminar duplicados
df_combinado.drop(columns=(["player_id"]), inplace=True)
# Guardar archivo preparado para subir
df_combinado.to_csv("data/jugadores_info_normalizado.csv", index=False)

* Lesiones

In [ ]:
# Importar librerías
import pandas as pd
from rapidfuzz import process, fuzz
import src as src
pd.set_option('display.max_columns', None)
# Cargar archivos
df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")
df_lesiones = pd.read_csv("data/lesiones_completo.csv")
# Crear listas nombres únicos
df_jugadoresunicos_eventos = df_eventos["player"].unique().tolist()
df_jugadoresunicos = df_lesiones["href"].unique().tolist()
df_clubesunicos_eventos = df_eventos["team"].unique().tolist()
df_clubesunicos = df_lesiones["equipo"].unique().tolist()
# Aplicar función normalización
df_nombres = src.normalizacion_nombres(df_jugadoresunicos, df_jugadoresunicos_eventos)
df_clubes = src.normalizacion_nombres(df_clubesunicos, df_clubesunicos_eventos)
# Crear diccionario
dict_nombres_info = df_nombres.set_index('Nombre_original')['Nombre_unificado'].to_dict()
# Mapear nombres
df_lesiones["href"] = df_lesiones["href"].apply(
    lambda x: dict_nombres_info.get(x, x) if dict_nombres_info.get(x) is not None else x)
# Crear diccionario para mapear los equipos
df_equipos_jugadores = df_eventos[["player", "team"]].drop_duplicates()
dict_equipos_jugadores = dict(zip(df_equipos_jugadores["player"], df_equipos_jugadores["team"]))
# Mapear los equipos en la temporada 2015/2016 ya que la info que aparece es actual
df_lesiones["equipo"] = df_lesiones["href"].map(dict_equipos_jugadores)
# Crear diccionarios para mapear los IDs de jugador y equipo
df_equipos = df_eventos[["team", "team_id"]].drop_duplicates()
dict_equipos = dict(zip(df_equipos["team"], df_equipos["team_id"]))
df_jugadores = df_eventos[["player", "player_id"]].drop_duplicates()
dict_jugadores = dict(zip(df_jugadores["player"], df_jugadores["player_id"]))
# Mapear ID Jugadores
df_lesiones["playerID"] = df_lesiones["href"].map(dict_jugadores)
# Mapear ID Equipos
df_lesiones["clubID"] = df_lesiones["equipo"].map(dict_equipos)
# Crear unicos para lesiones
tipo_lesiones_unico = df_lesiones["TIPO"].unique().tolist()
subtipo_lesiones_unico = df_lesiones["SUB TIPO"].unique().tolist()
dict_subtipo = {}
item = 0
for x in subtipo_lesiones_unico:
    item = item + 1
    dict_subtipo[x] = item

dict_tipo = {}
item_1 = 1
for x in tipo_lesiones_unico:
    item_1 = item_1 + 1
    dict_tipo[x] = item_1

# Mapear nombres unificados
df_lesiones["TIPO_ID"] = df_lesiones["TIPO"].map(dict_tipo)
df_lesiones["SUB_TIPO_ID"] = df_lesiones["SUB TIPO"].map(dict_subtipo)

# Guardar archivo
df_lesiones.to_csv("data/lesiones_completo_normalizado.csv", index=False)

* Datos Equipos

In [ ]:
# Importar librerías
import pandas as pd
from statsbombpy import sb
from rapidfuzz import process, fuzz
import src as src
# Cargar datos y archivos
df_statsbomb = sb.matches(competition_id=11, season_id = 27)
df_wyscout = pd.read_excel("data/Team Stats Consolidado.xlsx")
# Crear listas unicas
clubs_statsbomb = df_statsbomb["home_team"].unique().tolist()
clubs_wyscout = df_wyscout["Local"].unique().tolist()
clubs_wyscout_stats = df_wyscout["Stats Equipo"].unique().tolist()
# Eliminar sufijos columnas
df_wyscout['Visitante'] = df_wyscout['Visitante'].str.strip()
df_wyscout['Local'] = df_wyscout['Local'].str.strip()
df_wyscout['Stats Equipo'] = df_wyscout['Stats Equipo'].str.strip()
# Normalizar nombres
df_clubes = src.normalizacion_nombres(clubs_statsbomb, clubs_wyscout)
df_clubes_stats = src.normalizacion_nombres(clubs_wyscout_stats, clubs_statsbomb)
# Crear diccionario
dict_nombres_equipos = dict(zip(df_clubes["Nombre_unificado"], df_clubes["Nombre_original"]))
# Mapear nombres en todas las columnas
df_wyscout["Local"] = df_wyscout["Local"].map(dict_nombres_equipos)
df_wyscout["Visitante"] = df_wyscout["Visitante"].map(dict_nombres_equipos)
# Crear diccionario
dict_nombres_equipos_stats = dict(zip(df_clubes_stats["Nombre_original"], df_clubes_stats["Nombre_unificado"]))
# Mapear nombres en todas las columnas
df_wyscout["Stats Equipo"] = df_wyscout["Stats Equipo"].map(dict_nombres_equipos_stats)
# Unir df
df_combinado = df_wyscout.merge(
    df_statsbomb[["home_team", "away_team", "match_id"]],
    left_on=["Local", "Visitante"],
    right_on=["home_team", "away_team"],
    how="left")
# Guardar archivos
df_combinado.to_csv("data/Team_Stats_Normalizado.csv", index=False)

#### Carga Datos BBDD SQL

* Clasificación

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
from src import extraer_coordenadas_manual, insertar_datos_automatico, conexion_postgres
import psycopg2
pd.set_option('display.max_columns', None)
import os
from dotenv import load_dotenv
import requests
# Cargar archivo
df_clasificacion = pd.read_csv("data/clasificacion.csv")
# Eliminar duplicados
df_clasificacion.drop_duplicates(inplace = True)
# Hacer index
df_clasificacion["index"] = df_clasificacion["Jornada"].astype(str) + "_" + df_clasificacion["clubid"].astype(str)
# Renombrar columnas
df_clasificacion.rename(columns = ({
    "Pts" : "Puntos",
    "P" : "Partidos",
    "V" : "Victorias",
    "E" : "Empates",
    "D" : "Derrotas",
    "GM" : "Goles_a_favor",
    "GR" : "Goles_en_contra",
    "DG" : "Diferencia_goles"
}), inplace = True)
# Carga BBDD
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_clasificacion,"clasficacion", cur, conn)

* Equipos

In [ ]:
# Importar librerías
import pandas as pd
import os
from dotenv import load_dotenv
import requests
from src import insertar_datos_automatico, conexion_postgres
pd.set_option('display.max_columns', None)
import psycopg2
# Cargar archivos
df_completo = pd.read_csv("data/jugadores_info_normalizado.csv")
# Eliminar duplicados
df_clubs = df_completo[["club", "clubID"]].drop_duplicates().dropna().reset_index().drop(columns = ["index"])
# Carga BBDD
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_clubs,"clubes", cur, conn)

* Eventos

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
from src import extraer_coordenadas_manual, insertar_datos_automatico, conexion_postgres
import psycopg2
pd.set_option('display.max_columns', None)
import os
from dotenv import load_dotenv
import requests
# Cargar archivo
df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")
# Crear Id por tipo de evento
dict_id_tipo_evento = {}
id_tipo_evento = 1
for x in df_eventos["type"].unique().tolist():
    dict_id_tipo_evento[x] = id_tipo_evento
    id_tipo_evento = id_tipo_evento + 1
df_eventos["type_id"] = df_eventos["type"].map(dict_id_tipo_evento)
# Eliminar columnas innecesarias
df_eventos.drop(columns=(['bad_behaviour_card',
 'off_camera',
 'related_events',
 'substitution_outcome',
 'substitution_outcome_id',
 'substitution_replacement',
 'substitution_replacement_id',
 'shot_freeze_frame',
 '50_50',
 'index',
 'duration',
 'tactics',
 "pass_angle",
 "pass_length",
 "minute",
 "second",
 'injury_stoppage_in_chain',
 'player_off_permanent',
 'half_end_early_video_end']), inplace=True)

# Crear columnas coordenadas
lista_col_location = []
for col in df_eventos.columns:
    if "location" in col:
        lista_col_location.append(col)
for loc_col_name in lista_col_location:
    # Nombres para las nuevas columnas de x e y
    nueva_col_x = f"{loc_col_name}_x"
    nueva_col_y = f"{loc_col_name}_y"
    nueva_col_z = f"{loc_col_name}_z"
    # Aplicar la función y asignar a las nuevas columnas
    df_eventos[[nueva_col_x, nueva_col_y, nueva_col_z]] = df_eventos[loc_col_name].apply( lambda val: pd.Series(extraer_coordenadas_manual(val)))
lista_drop = []
for drop in df_eventos.columns:
    if drop.startswith("shot"):
        continue
    if "location" in drop and drop.endswith("z"):
        lista_drop.append(drop)
df_eventos.drop(columns=lista_drop, inplace=True)
df_eventos.drop(columns=lista_col_location, inplace=True)

# Modificar columnas booleanas y numericas con NaN para pasar a None
columnas_bool = []

for col in df_eventos.columns:
    if df_eventos[col].apply(lambda x: isinstance(x, bool) and x is True).any():
        df_eventos[col] = df_eventos[col].fillna(False)
        columnas_bool.append(col)

lista_cols_int = ["match_id", "pass_recipient_id", "period", "player_id", "possession", "possession_team_id", "team_id", "type_id"]
for x in lista_cols_int:
    df_eventos[x] = df_eventos[x].astype('Int64')
    df_eventos[x].replace({pd.NA: None}, inplace=True)

df_eventos.replace({pd.NA: None}, inplace=True)

# Renombrar columnas
df_eventos.rename(columns=({"player_id":"playerid",
                            "player": "nombre_completo_jugador",
                            "team_id": "clubid",
                            "team": "club"}), inplace=True)

# Carga BBDD
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_eventos,"eventos", cur, conn)

* Jugadores

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests
from src import insertar_datos_automatico, conexion_postgres, convertir_fechaeventos, combinar
pd.set_option('display.max_columns', None)
import psycopg2
# Carga archivos
df_jugadores = pd.read_csv("data/jugadores_info_normalizado.csv")
# Seleccionar columnas válidas
df_jugadores_columnasok = df_jugadores[['playerID', 'playerName', 'playerFullName', 'birthplace',
       'dateOfBirth', 'birthplaceCountry', 'age', 'height', 'foot', 'internationalTeam',
       'internationalGames', 'internationalGoals', 'country', 'secondCountry', 'club',
       'clubID', 'playerMainPosition', 'playerSecondPosition',
       'playerThirdPosition', 'matches_played', 'starts', 'min', 'gls',
       'ast', 'non_pen_gls', 'yellow_cards', 'red_cards',
       'per90_gls', 'per90_ast', 'per90_non_pen_gls', 'subs',]]
# Renombrar columnas
df_jugadores_columnasok.rename(columns = {
    'playerName': 'Nombre_Jugador',
    'playerFullName': 'Nombre_Completo_Jugador',
    'birthplace': 'Lugar_Nacimiento',
    'dateOfBirth': 'Fecha_Nacimiento',
    'birthplaceCountry': 'Pais_Nacimiento',
    'age': 'Edad',
    'height': 'Altura',
    'foot': 'Pie_Dominante',
    'internationalTeam': 'Equipo_Internacional',
    'internationalGames': 'Partidos_Internacionales',
    'internationalGoals': 'Goles_Internacionales',
    'country': 'Pais',
    'secondCountry': 'Segundo_Pais',
    'club': 'Club',
    'playerMainPosition': 'Posicion_Principal',
    'playerSecondPosition': 'Posicion_Secundaria',
    'playerThirdPosition': 'Posicion_Terciaria',
    'matches_played' : 'Partidos_jugados', 
    'starts': 'Titularidades', 
    'min': 'Minutos', 
    'gls': 'Goles',
    'ast': 'Asistencias',
    'non_pen_gls': 'Goles_sin_penaltis',
    'yellow_cards': 'Amarillas',
    'red_cards': 'Rojas',
    'per90_gls': 'Goles_por_90',
    'per90_ast': 'Asistencias_por_90',
    'per90_non_pen_gls': 'Goles_sin_penaltis_por_90',
    'subs': 'Suplencias'
}, inplace = True)
# Quitar duplicados y cambiar tipos de dato
df_jugadores_columnasok = df_jugadores_columnasok.dropna(subset=["playerID"])
df_jugadores_columnasok["Altura"].replace("indeterminado", np.nan, inplace=True)
df_jugadores_columnasok["Altura"] = df_jugadores_columnasok["Altura"].str.replace(",", ".").astype(float)
df_jugadores_columnasok["Fecha_Nacimiento"] = df_jugadores_columnasok["Fecha_Nacimiento"] = pd.to_datetime(df_jugadores_columnasok["Fecha_Nacimiento"])
df_jugadores_columnasok["playerID"] = df_jugadores_columnasok["playerID"].astype(int)
df_jugadores_columnasok["clubID"] = df_jugadores_columnasok["clubID"].astype(int)
df_jugadores_columnasok["Partidos_Internacionales"] = df_jugadores_columnasok["Partidos_Internacionales"].astype('Int64')
df_jugadores_columnasok["Goles_Internacionales"] = df_jugadores_columnasok["Goles_Internacionales"].astype('Int64')
df_jugadores_columnasok["Titularidades"] = df_jugadores_columnasok["Titularidades"].astype('Int64')
df_jugadores_columnasok["Goles"] = df_jugadores_columnasok["Goles"].astype('Int64')
df_jugadores_columnasok["Asistencias"] = df_jugadores_columnasok["Asistencias"].astype('Int64')
df_jugadores_columnasok["Amarillas"] = df_jugadores_columnasok["Amarillas"].astype('Int64')
df_jugadores_columnasok["Rojas"] = df_jugadores_columnasok["Rojas"].astype('Int64')
# Unificar filas duplicadas
indices = df_jugadores_columnasok[df_jugadores_columnasok["playerID"].duplicated()].index
for x in indices:
    fila_actual = df_jugadores_columnasok.loc[x]
    fila_anterior = df_jugadores_columnasok.loc[x - 1]
    # Aseguramos tipos escalar combinando columna por columna
    fila_unificada = pd.Series(
        {col: combinar(fila_actual[col], fila_anterior[col]) for col in df_jugadores_columnasok.columns})
    # Reemplazar la fila anterior con la fila unificada
    df_jugadores_columnasok.loc[x - 1] = fila_unificada
# Eliminar filas duplicadas
df_jugadores_columnasok = df_jugadores_columnasok.drop(index=indices).reset_index(drop=True)
# Cambiar Nan por None
df_jugadores_columnasok = df_jugadores_columnasok.astype(object).where(pd.notna(df_jugadores_columnasok), None)
# Cargar BBDD
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_jugadores_columnasok,"jugadores", cur, conn)



* Partidos

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests
from src import insertar_datos_automatico, conexion_postgres
pd.set_option('display.max_columns', None)
import psycopg2
# Cargar archivos
df_partidos = pd.read_csv("data/Team_Stats_Normalizado.csv")
df_completo = pd.read_csv("data/jugadores_info_normalizado.csv")
df_clubs = df_completo[["club", "clubID"]].drop_duplicates().dropna().reset_index().drop(columns = ["index"])
# Eliminar columnas
df_partidos.drop(columns = ["home_team", "away_team"], inplace = True)
# Crear diccionario para mapear nombres y los Ids de cada equipo
diccionario_clubs = dict(zip(df_clubs["club"], df_clubs["clubID"]))
df_partidos["id_club_local"] = df_partidos["Local"].map(diccionario_clubs)
df_partidos["id_club_visitante"] = df_partidos["Visitante"].map(diccionario_clubs)
df_partidos["id_club_stats"] = df_partidos["Stats Equipo"].map(diccionario_clubs)
# Renombrar columnas
df_partidos.rename(columns = {'Seleccionar esquema': 'Esquema'}, inplace = True)
df_partidos.columns = df_partidos.columns.str.replace(' ', '_')
df_prueba = df_partidos.copy()
# Creamos una clave única por partido
df_prueba['match_id'] = df_prueba['match_id'].astype(int)  # asegúrate de que sea tipo entero si no lo está
# Separar columnas fijas (que no queremos duplicar) de métricas
columnas_clave = ['Fecha', 'Partido', 'Resultado', 'match_id', 'Local', 'Visitante','Competición', 'Duración', 'id_club_local',  'id_club_visitante', 'id_club_stats']
columnas_metricas = df_prueba.columns.difference(columnas_clave).tolist()

# Pivotar el DataFrame para tener columnas por equipo
df_local = df_prueba[df_partidos['Local'] == df_prueba['Stats_Equipo']].copy()
df_visitante = df_prueba[df_partidos['Visitante'] == df_prueba['Stats_Equipo']].copy()

# Renombramos columnas
df_local = df_local.set_index('match_id')
df_local.columns = [col + '_local' if col not in columnas_clave else col for col in df_local.columns]

df_visitante = df_visitante.set_index('match_id')
df_visitante.columns = [col + '_visitante' if col not in columnas_clave else col for col in df_visitante.columns]

# Unimos los dos DataFrames
df_final = df_local.join(df_visitante,lsuffix='_L', rsuffix='_V', how='inner').reset_index()
#Eliminar columnas no necesarias
df_final.drop(columns=([ 'Stats_Equipo_local',
 'id_club_stats_L',
 'Fecha_V',
 'Partido_V',
 'Resultado_V',
 'Local_V',
 'Visitante_V',
 'Competición_V',
 'Duración_V',
 'Tiros_libres_con_remate.1_local',
 'Tiros_libres_con_remate.1_visitante',
 'Stats_Equipo_visitante',
 'id_club_stats_V',
 'id_club_local_V',
 'id_club_visitante_V',
 'Jornada_local']), inplace=True)
# Renombrar columnas
df_final.rename(columns={'Fecha_L':'Fecha',
 'Partido_L':'Partido',
 'Resultado_L': 'Resultado',
 'Local_L':'Local',
 'Visitante_L':'Visitante',
 'Competición_L':'Competición',
 'Duración_L':'Duración',
 'Jornada_visitante':'Jornada',
 'id_club_local_L' :'id_club_local',
 'id_club_visitante_L' :'id_club_visitante',
}, inplace=True)
df_final.columns = df_final.columns.str.replace('%', 'porcentaje')
df_final.columns = df_final.columns.str.replace(',', '')
# Cargar BBDD
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_final,"partidos", cur, conn)



* Lesiones

In [ ]:
# Importar librerías
import pandas as pd
pd.set_option('display.max_columns', None)
import os
from dotenv import load_dotenv
import requests
from src import insertar_datos_automatico, conexion_postgres, convertir_fechaeventos
import psycopg2
import datetime
# Cargar archivo
df_lesiones = pd.read_csv("data/lesiones_completo_normalizado.csv")
# Renombrar columnnas y cambiar tipos de dato
df_lesiones.rename(columns = {
    'JUGADOR': 'Nombre_Jugador',
    'href': 'Nombre_Completo_Jugador',
    'equipo': 'Club',
    'TIPO': 'Tipo_Lesion',
    'SUB TIPO': 'Sub_Tipo_Lesion',
    'TIPO_ID': 'Tipo_Lesion_ID',
    'SUB_TIPO_ID': 'Sub_Tipo_Lesion_ID',
    'DATA': 'Fecha_incio_lesion',
    'FIM': 'Fecha_fin_lesion'
}, inplace = True)
df_lesiones = convertir_fechaeventos(df_lesiones)
df_lesiones = df_lesiones.dropna(subset=["playerID"])
df_lesiones["playerID"] = df_lesiones["playerID"].astype(int)
df_lesiones["clubID"] = df_lesiones["clubID"].astype(int)
# Elegir unicamente lesiones
df_lesiones = df_lesiones[df_lesiones["Tipo_Lesion"] == "Injury"]
# Quitar columnas no necesarias
df_lesiones.drop(columns=(["Tipo_Lesion_ID", "Tipo_Lesion"]), inplace=True)
# Renombrar
df_lesiones.rename(columns = {
    'Sub_Tipo_Lesion': 'Tipo_Lesion',
    'Sub_Tipo_Lesion_ID': 'Tipo_Lesion_ID',
}, inplace = True)
# Crear tabla indice
tabla_indice_lesiones = df_lesiones[["Tipo_Lesion", "Tipo_Lesion_ID"]].drop_duplicates()
df_lesiones["ID_Lesion"] = df_lesiones["Nombre_Jugador"].index
# Modificar nombres para powerBI y agrupar en 15 Ids
mapa_lesiones_grupo = {
    # 1. Afecciones Abdominales y Gastrointestinales
    29: "Afecciones Abdominales y Gastrointestinales",  # Stomach Problems
    62: "Afecciones Abdominales y Gastrointestinales",  # Abdominal Problems

    # 2. Afecciones del Tendón de Aquiles
    33: "Afecciones del Tendón de Aquiles",  # Achilles tendon rupture
    44: "Afecciones del Tendón de Aquiles",  # Achilles Heel Problems
    45: "Afecciones del Tendón de Aquiles",  # Achilles tendon problems
    61: "Afecciones del Tendón de Aquiles",  # Achilles Tendon Irritation

    # 3. Lesiones de Cadera
    54: "Lesiones de Cadera",  # Hip Injury
    65: "Lesiones de Cadera",  # Hip Bruise

    # 4. Lesiones de Cabeza, Cara y Cuello
    13: "Lesiones de Cabeza, Cara y Cuello",  # Neck Injury
    57: "Lesiones de Cabeza, Cara y Cuello",  # Cheekbone Fracture
    68: "Lesiones de Cabeza, Cara y Cuello",  # Facial Fracture

    # 5. Lesiones de Espalda
    55: "Lesiones de Espalda",  # Back Injury

    # 6. Lesiones de Hombro y Clavícula
    46: "Lesiones de Hombro y Clavícula",  # Broken Collarbone
    70: "Lesiones de Hombro y Clavícula",  # Shoulder Injury

    # 7. Lesiones de Ingle, Aductores y Abductores
    3:  "Lesiones de Ingle, Aductores y Abductores",  # Adductor Pain
    6:  "Lesiones de Ingle, Aductores y Abductores",  # Tear in the abductor muscle
    15: "Lesiones de Ingle, Aductores y Abductores",  # Groin Injury
    38: "Lesiones de Ingle, Aductores y Abductores",  # Adductor Injury
    53: "Lesiones de Ingle, Aductores y Abductores",  # Groin Problems

    # 8. Lesiones de Ligamentos (Generales/No Articulares)
    31: "Lesiones de Ligamentos (Generales/No Articulares)",  # Ligament Stretching
    32: "Lesiones de Ligamentos (Generales/No Articulares)",  # Outer Ligament Problems

    # 9. Lesiones de Mano y Dedos
    47: "Lesiones de Mano y Dedos",  # Hand Injury
    50: "Lesiones de Mano y Dedos",  # Thumb Injury

    # 10. Lesiones de Muslo e Isquiotibiales
    28: "Lesiones de Muslo e Isquiotibiales",  # Hamstring Injury
    30: "Lesiones de Muslo e Isquiotibiales",  # Hamstring Strain
    34: "Lesiones de Muslo e Isquiotibiales",  # Thigh Problems
    67: "Lesiones de Muslo e Isquiotibiales",  # Thigh Muscle Strain

    # 11. Lesiones de Pierna Inferior (Pantorrilla y Huesos)
    2:  "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Calf Injury
    11: "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Calf Problems
    35: "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Broken Fibula
    41: "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Broken Leg
    42: "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Leg Injury

    # 12. Lesiones de Rodilla (Complejo y Contusiones)
    10: "Lesiones de Rodilla (Complejo y Contusiones)",  # Knee Problems
    18: "Lesiones de Rodilla (Complejo y Contusiones)",  # Knee Bruise
    36: "Lesiones de Rodilla (Complejo y Contusiones)",  # Knee Medial Ligament Tear
    37: "Lesiones de Rodilla (Complejo y Contusiones)",  # Knee Injury
    48: "Lesiones de Rodilla (Complejo y Contusiones)",  # Cruciate Ligament Injury
    49: "Lesiones de Rodilla (Complejo y Contusiones)",  # Meniscus Tear
    52: "Lesiones de Rodilla (Complejo y Contusiones)",  # Cruciate Ligament Tear
    58: "Lesiones de Rodilla (Complejo y Contusiones)",  # Twisted knee
    59: "Lesiones de Rodilla (Complejo y Contusiones)",  # Meniscus Injury

    # 13. Lesiones de Tobillo, Pie y Dedos
    4:  "Lesiones de Tobillo, Pie y Dedos",  # Ankle Problems
    12: "Lesiones de Tobillo, Pie y Dedos",  # Foot Injury
    22: "Lesiones de Tobillo, Pie y Dedos",  # Ankle Injury
    26: "Lesiones de Tobillo, Pie y Dedos",  # Toe Injury
    27: "Lesiones de Tobillo, Pie y Dedos",  # Ankle Sprain
    40: "Lesiones de Tobillo, Pie y Dedos",  # Broken Toe

    # 14. Lesiones Torácicas (Costillas, Pulmón)
    25: "Lesiones Torácicas (Costillas, Pulmón)",  # Lung Contusion
    43: "Lesiones Torácicas (Costillas, Pulmón)",  # Bruised Ribs

    # 15. Otros y Afecciones Generales
    5:  "Otros y Afecciones Generales",  # Rest
    7:  "Otros y Afecciones Generales",  # Muscle Strain
    8:  "Otros y Afecciones Generales",  # Muscle Injury
    14: "Otros y Afecciones Generales",  # Torn Muscle Bundle
    16: "Otros y Afecciones Generales",  # Knock
    19: "Otros y Afecciones Generales",  # Muscular problems
    20: "Otros y Afecciones Generales",  # Torn Muscle Fiber
    21: "Otros y Afecciones Generales",  # Minor Knock
    39: "Otros y Afecciones Generales",  # Fitness
    51: "Otros y Afecciones Generales",  # Infection
    60: "Otros y Afecciones Generales",  # Flu
    63: "Otros y Afecciones Generales",  # Virus
    64: "Otros y Afecciones Generales",  # Fever
    69: "Otros y Afecciones Generales"   # Unknown Injury
}
tabla_indice_lesiones["Tipo_Lesion"] = tabla_indice_lesiones["Tipo_Lesion_ID"].map(mapa_lesiones_grupo)
mapa_nombre_grupo_a_nuevo_id = {
    "Afecciones Abdominales y Gastrointestinales": 1,
    "Afecciones del Tendón de Aquiles": 2,
    "Lesiones de Cadera": 3,
    "Lesiones de Cabeza, Cara y Cuello": 4,
    "Lesiones de Espalda": 5,
    "Lesiones de Hombro y Clavícula": 6,
    "Lesiones de Ingle, Aductores y Abductores": 7,
    "Lesiones de Ligamentos (Generales/No Articulares)": 8,
    "Lesiones de Mano y Dedos": 9,
    "Lesiones de Muslo e Isquiotibiales": 10,
    "Lesiones de Pierna Inferior (Pantorrilla y Huesos)": 11,
    "Lesiones de Rodilla (Complejo y Contusiones)": 12,
    "Lesiones de Tobillo, Pie y Dedos": 13,
    "Lesiones Torácicas (Costillas, Pulmón)": 14,
    "Otros y Afecciones Generales": 15
}

tabla_indice_lesiones["Tipo_Lesion_ID"] = tabla_indice_lesiones["Tipo_Lesion"].map(mapa_nombre_grupo_a_nuevo_id)
tabla_indice_lesiones = tabla_indice_lesiones.groupby("Tipo_Lesion_ID").count().reset_index()
mapa_nuevo_id = {
   1: "Abdomen izquierdo",
    2: "Pantorrilla derecha",
    3: "Glúteo izquierdo",
    4: "Cabeza delantera",
    5: "Erector de la columna izquierdo",
    6: "Deltoide izquierdo delantero",
    7: "Aductor largo izquierdo",
    8: "Recto femoral izquierdo",
    9: "Flexor Digitorum Izquierdo",
    10: "Bíceps femoral izquierdo",
    11: "Pantorrilla izquierda",
    12: "Rodilla izquierda delantera",
    13: "Tibial anterior izquierdo",
    14: "Erector de la columna derecho",
    15: "Cabeza trasera"
}
tabla_indice_lesiones["Tipo_Lesion"] = tabla_indice_lesiones["Tipo_Lesion_ID"].map(mapa_nuevo_id)
df_lesiones["Tipo_Lesion"] = df_lesiones["Tipo_Lesion_ID"].map(mapa_lesiones_grupo)
df_lesiones["Tipo_Lesion_ID"] = df_lesiones["Tipo_Lesion"].map(mapa_nombre_grupo_a_nuevo_id)
df_lesiones["Tipo_Lesion"] = df_lesiones["Tipo_Lesion_ID"].map(mapa_nuevo_id)
df_lesiones["Tipo_Lesion"].unique()
# Cargar BBDD
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_lesiones,"lesiones", cur, conn)
insertar_datos_automatico(tabla_indice_lesiones,"Lesiones_indice", cur, conn)

#### EDA

In [ ]:
# Importar librerías y crear conexión con la BBDD
import pandas as pd
import numpy as np
import psycopg2
import src as src
from src import conexion_postgres
import os
from dotenv import load_dotenv
import re
import time
pd.set_option('display.max_columns', None)
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)

* Estadísticas partidos

In [ ]:
# Traer datos y hacer df
cur.execute("SELECT * FROM partidos")
datos_partidos = cur.fetchall()
columnas_partidos = [desc[0] for desc in cur.description]
df_partidos = pd.DataFrame(datos_partidos, columns=columnas_partidos)

In [ ]:
# Modificar df para ver columnas numericas y agupar por Athletic Club vs Liga
numericas = df_partidos.select_dtypes(include=np.number).columns.tolist()
df_numericas = df_partidos[numericas]
columnas_id_club_local = ["id_club_local"]
columnas_finales_local = list(set(numericas + columnas_id_club_local))
df_final_local = df_partidos[columnas_finales_local]
columnas_id_club_local_1 = df_final_local.columns.tolist()
columnas_id_club_local_1_ord = sorted(columnas_id_club_local_1)
df_final_local_ordenado = df_final_local[columnas_id_club_local_1_ord]
columnas_id_club_visitante = ["id_club_visitante"]
columnas_finales_visitante = list(set(numericas + columnas_id_club_visitante))
df_final_visitante = df_partidos[columnas_finales_visitante]
columnas_id_club_visitante_1 = df_final_visitante.columns.tolist()
columnas_id_club_visitante_1 = sorted(columnas_id_club_visitante_1)
df_final_visitante_ordenado = df_final_visitante[columnas_id_club_visitante_1]
df_final_local_ordenado_medias = df_final_local_ordenado.groupby("id_club_local").mean().reset_index()
ath_local = df_final_local_ordenado_medias.iloc[9]
mean_row_local = df_final_local_ordenado_medias.mean(numeric_only=True)
for col in df_final_local_ordenado_medias.columns:
    if col not in mean_row_local:
        mean_row_local[col] = None  # o np.nan
mean_row_ord = mean_row_local[ath_local.index]
df_ath_local = pd.DataFrame([ath_local, mean_row_ord])
df_ath_local.replace(pd.NA, "Local", inplace=True)
df_ath_local.rename(columns=({"id_club_local":"id_club"}), inplace=True)
mean_row_local = df_final_local_ordenado_medias.mean(numeric_only=True)
for col in df_final_local_ordenado_medias.columns:
    if col not in mean_row_local:
        mean_row_local[col] = None
df_final_visitante_ordenado_medias = df_final_visitante_ordenado.groupby("id_club_visitante").mean().reset_index()
ath_visitante = df_final_visitante_ordenado_medias.iloc[9]
mean_row_visitante = df_final_visitante_ordenado_medias.mean(numeric_only=True)
for col in df_final_visitante_ordenado_medias.columns:
    if col not in mean_row_visitante:
        mean_row_visitante[col] = None  # o np.nan
mean_row_visitante_ord = mean_row_visitante[ath_visitante.index]
df_ath_visitante = pd.DataFrame([ath_visitante, mean_row_visitante_ord])
df_ath_visitante.replace(pd.NA, "Visitante", inplace=True)
df_ath_visitante.rename(columns=({"id_club_visitante":"id_club"}), inplace=True)
# Ver datos como local
df_ath_local
# Ver datos como visitante
df_ath_visitante
# Unir a datos como local
df_ath_vs_liga = pd.concat([df_ath_local, df_ath_visitante])
# Datos vs Liga
df_ath_vs_liga

* Lesiones

In [ ]:
# Traer datos
cur.execute("SELECT * FROM lesiones")
datos_lesiones = cur.fetchall()
columnas_lesiones = [desc[0] for desc in cur.description]
df_lesiones = pd.DataFrame(datos_lesiones, columns=columnas_lesiones)
# Crear df general y uno del equipo
prueba = df_lesiones.groupby(["club", "tipo_lesion_id"])["playerid"].count().reset_index()
prueba_ath = df_lesiones[df_lesiones["club"] == "Athletic Club"]
ath_lesiones = prueba_ath.groupby("tipo_lesion")["playerid"].count().reset_index()
ath_lesiones.rename(columns = {"playerid":"n_lesiones"}, inplace=True)
# Nº de lesiones
ath_lesiones
# Comparar con resto de la liga
liga_lesiones = df_lesiones.groupby("tipo_lesion")["playerid"].count().reset_index()
liga_lesiones.rename(columns = {"playerid":"n_lesiones"}, inplace=True)
liga_lesiones = liga_lesiones.groupby("tipo_lesion").mean().reset_index()
liga_lesiones
comparacion_lesiones = pd.merge(liga_lesiones, ath_lesiones, how= "left", on='tipo_lesion')
comparacion_lesiones.rename(columns = {"n_lesiones_x":"lesiones_liga", "n_lesiones_y":"lesiones_ath"}, inplace=True)
comparacion_lesiones = comparacion_lesiones.sort_values(by='lesiones_ath', ascending=False)
comparacion_lesiones.head(15)
# Ver lesiones por jugador
prueba_ath.groupby(["nombre_jugador", "tipo_lesion"])["tipo_lesion_id"].count().reset_index()

* Jugadores

In [ ]:
# Carga datos
cur.execute("SELECT * FROM jugadores")
datos_jugadores = cur.fetchall()
columnas_jugadores = [desc[0] for desc in cur.description]
df_jugadores = pd.DataFrame(datos_jugadores, columns=columnas_jugadores)
# Hacer df para el equipo
df_jugadores_ath = df_jugadores[df_jugadores["club"] == "Athletic Club"]
# Cambiar fechas
df_jugadores_ath["fecha_nacimiento"] = pd.to_datetime(df_jugadores_ath["fecha_nacimiento"], errors='coerce')
df_jugadores["fecha_nacimiento"] = pd.to_datetime(df_jugadores["fecha_nacimiento"], errors='coerce')
# Crear edad en 2015
df_jugadores["edad_liga"] = df_jugadores["fecha_nacimiento"].dt.year
df_jugadores_ath["edad_liga"] = df_jugadores_ath["fecha_nacimiento"].dt.year
# Visualizar datos tanto de equipo como de la Liga en general
df_jugadores_ath.describe()
df_jugadores.describe()